In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import QuantileTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import class_weight
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [2]:
# Read the CSV file
df = pd.read_csv("resources/pollution_dataset.csv")

# Review the DataFrame
df.head(5)

,Temperature,Humidity,PM2.5,PM10,NO2,SO2,CO,Proximity_to_Industrial_Areas,Population_Density,Air Quality
0,27.2,51.7,35.1,46.2,26.7,32.2,0.98,11.2,314,Hazardous
1,26.3,59.3,1.0,6.2,38.3,20.4,0.68,13.5,298,Good
2,27.9,73.2,20.0,39.4,19.6,5.8,0.95,5.4,309,Good
3,23.9,51.9,14.7,24.3,5.2,12.6,1.24,4.5,282,Poor
4,25.2,59.0,26.3,30.9,26.8,13.5,1.06,5.6,293,Poor


In [3]:
num_entries = len(df)
print(f"Number of entries: {num_entries}")

Number of entries: 5000


In [4]:
# Check for missing values
missing_values = df.isnull().sum()
print(missing_values)

Temperature                      0
Humidity                         0
PM2.5                            0
PM10                             0
NO2                              0
SO2                              0
CO                               0
Proximity_to_Industrial_Areas    0
Population_Density               0
Air Quality                      0
dtype: int64


In [6]:
df_cleaned = df.drop(columns=["Temperature", "Humidity", "Proximity_to_Industrial_Areas", "Population_Density"])

## Find and remove outliers

In [7]:
df_cleaned.columns

Index(['PM2.5', 'PM10', 'NO2', 'SO2', 'CO', 'Air Quality'], dtype='object')

In [ ]:
# Create a dictionary to hold outliers
outliers_dict = {}

In [ ]:
# Define a function to detect outliers using the IQR method
def detect_outliers_iqr(data):
    outliers_dict = {}
    for column in data.columns:
        if column != 'air_quality_encoded':  # Exclude the target variable
            Q1 = data[column].quantile(0.25)
            Q3 = data[column].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
            outliers_dict[column] = outliers
    return outliers_dict

In [ ]:
# Detect outliers in the DataFrame
outliers_dict = detect_outliers_iqr(df)

In [ ]:
# Print outliers for each column (excluding air_quality_encoded)
for column, outliers in outliers_dict.items():
    print(f"Outliers in column '{column}':")
    print(outliers)
    print("\n")

In [ ]:
outliers_dict[column] = outliers

In [ ]:
# Define a function to remove outliers using the IQR method

def remove_outliers_iqr(data):
    for column in data.columns:
        if column != 'air_quality_encoded':  # Exclude the target variable
            Q1 = data[column].quantile(0.25)
            Q3 = data[column].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            # Remove outliers
            data = data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]
    return data

In [ ]:
# Confirm outliers were removed
print(f"Original DataFrame shape: {df.shape}")
print(f"Cleaned DataFrame shape: {cleaned_df.shape}")

In [ ]:
cleaned_df = remove_outliers_iqr(df)

In [8]:
X = df_cleaned.drop('Air Quality', axis=1)  # Features
y = df_cleaned['Air Quality']  # Target

In [9]:
# Initialize the LabelEncoder
label_encoder = LabelEncoder()

In [10]:
# Fit and transform the target variable
y_encoded = label_encoder.fit_transform(y)

In [11]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

In [21]:
from tensorflow import keras
from tensorflow.keras import layers

In [22]:
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # Input layer
    layers.Dense(32, activation='relu'),  # Hidden layer
    layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])


C:\Users\hebir\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.1123 - loss: -8.9062 - val_accuracy: 0.1075 - val_loss: -54.0501
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0991 - loss: -109.7452 - val_accuracy: 0.1075 - val_loss: -371.4854
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1072 - loss: -613.1423 - val_accuracy: 0.1075 - val_loss: -1407.4409
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0984 - loss: -2227.5774 - val_accuracy: 0.1075 - val_loss: -3659.3137
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0923 - loss: -6053.3647 - val_accuracy: 0.1075 - val_loss: -7550.6357
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1039 - loss: -10635.9209 - val_accuracy: 0.1075 - val_loss: -13067.7188
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1020 - loss: -16210.8926 - val_accuracy: 0.1075 - val_loss: -20614.6406
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 

In [25]:
loss, accuracy = model.evaluate(X_test, y_test)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1019 - loss: -70880.6328  


In [26]:
print(f'Test Accuracy: {accuracy:.2f}')

Test Accuracy: 0.10


In [13]:
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)
rf_model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=500, random_state=78)

In [15]:
y_pred = rf_model.predict(X_test)

In [17]:
from sklearn.metrics import accuracy_score

In [18]:
accuracy = accuracy_score(y_test, y_pred)

In [19]:
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.39


In [ ]:
print("X_train_scaled shape:", X_train_scaled.shape)
print("y_train shape:", y_train.shape)

In [ ]:
# Apply SMOTE to the training data
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)  # Use the same scaler for the test data

In [ ]:
print("X_train_scaled shape:", X_train_scaled.shape)
print("y_train_resampled shape:", y_train_resampled.shape)

In [ ]:
rf_model = RandomForestClassifier()
rf_model.fit(X_train_scaled, y_train_resampled)  # Use y_train_resampled here

In [ ]:
# Predictions
y_pred = rf_model.predict(X_test_scaled)

In [ ]:
# Print the classification report
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
# Model 2: Logistic Regression
lr_model = LogisticRegression()
lr_model.fit(X_train_scaled, y_train_resampled)  # Use y_train_resampled her)

In [ ]:
y_pred_lr = lr_model.predict(X_test_scaled)

In [ ]:
print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_lr))

In [ ]:
# K-means Clustering
# Choose the number of clusters (k)
k = 4  # You can adjust this based on your specific use case

In [ ]:
# Initialize and fit the K-means model
kmeans_model = KMeans(n_clusters=k, random_state=42)
kmeans_model.fit(X_train_scaled)

In [ ]:
# Predict cluster labels for the training data
clusters = kmeans_model.predict(X_train_scaled)

In [ ]:
X_train_with_clusters = np.hstack((X_train_scaled, clusters.reshape(-1, 1)))

In [ ]:
# Print cluster centers
print("Cluster Centers:")
print(kmeans_model.cluster_centers_)

In [ ]:
# Dimensionality Reduction with PCA
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train_scaled)

In [ ]:
# Visualize the clusters
plt.figure(figsize=(10, 6))
plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=clusters, cmap='viridis', marker='o', edgecolor='k', s=50)
plt.title('K-means Clustering Visualization')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')

In [ ]:
# Plot cluster centers
centers = pca.transform(kmeans_model.cluster_centers_)
plt.scatter(centers[:, 0], centers[:, 1], c='red', marker='X', s=200, label='Cluster Centers')
plt.legend()
plt.grid()
plt.show()

## Check feature importance

In [ ]:
importances = model.feature_importances_

In [ ]:
# Create a DataFrame for feature importances
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

In [ ]:
plt.figure(figsize=(10, 6))
plt.barh(feature_importances['Feature'], feature_importances['Importance'], color='skyblue')
plt.xlabel('Importance')
plt.title('Feature Importance')
plt.gca().invert_yaxis()
plt.show()